## Create some local directories in /tmp
  * as a downloads cache

# Determine which T5 FLAN models we want to 
  * serialize
  * deserialize
  * run inference

In [1]:
import os
fsRoot = '/tmp/base'

zipRoot = fsRoot + '/root'
modelDest = zipRoot + '/saved' 
tokenDir = zipRoot + '/token'

import pathlib
dirs = [ modelDest, tokenDir ]
for d in dirs:
    pathlib.Path(d).mkdir(parents=True, exist_ok=True)

modelList = ["google/flan-t5-base"] ## Add larger models as required
#modelList = ["google/flan-t5-base", "google/flan-t5-large", "google/flan-t5-xl", "google/flan-t5-xxl" ]
# Model Sizes: 1GB (base), 3GB (large), 11GB (xl), 44GB (xxl)
# "google/flan-t5-base", "google/flan-t5-large", "google/flan-t5-xl", "google/flan-t5-xxl" 


In [2]:
import pandas as pd
from transformers import T5Tokenizer,T5ForConditionalGeneration, T5Config

import torch

In [3]:
for modelID in modelList:
    tokenizer = T5Tokenizer.from_pretrained(modelID, cache_dir=fsRoot)
    model = T5ForConditionalGeneration.from_pretrained(modelID, cache_dir=fsRoot)

    serialize=True
    if (serialize):
        tokenizer.save_pretrained(tokenDir) # huggingface API take directory
        torch.save(model.state_dict(), modelDest + '/pydict.pt') #torch API takes file
        config = T5Config.from_pretrained(modelID)
        config.to_json_file(modelDest + '/config.json')
        print ('wrote model to:',  modelDest + '/pydict.pt')
        print ('wrote config to:',  modelDest + '/config.json')
        print ('wrote tokenizer config to:',  tokenDir )

    inference = True
    if (inference):
        infile = './flan_prompt.csv' 
        df = pd.read_csv(infile)
        for p in list(df['prompt'].unique()):
            print (p)
            input_ids = tokenizer(p, return_tensors="pt").input_ids
            outputs = model.generate(input_ids ,max_length=50)
            response = tokenizer.decode(outputs[0])
            response = response.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
            print (response.strip())
            print ('-----------------------')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


wrote model to: /tmp/base/root/saved/pydict.pt
wrote config to: /tmp/base/root/saved/config.json
wrote tokenizer config to: /tmp/base/root/token
translate English to French: What time is it??
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Quelle temps est-ce?
-----------------------
translate English to German: What is your name?
Was ist Ihr Name?
-----------------------
Translate this English sentence to Spanish: Cat loves chicken pizza
El rato encanta la pizza de chile
-----------------------
Alice made her friend, Xin, laugh when she told a funny joke.\n\nWho is she referring to?\n\n(A)Alice \n(B)Xin
Alice
-----------------------
Alice made her friend, Xin, laugh when she told a funny joke.\n\nWho laughed?\n\n(A)Alice \n(B)Xin
Alice
-----------------------
Answer the following yes/no

# Done - ZZZZ